In [15]:
import pandas as pd
df = pd.read_csv('tmdb_5000_movies.csv')

In [16]:
df['genres'].head()

0    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1    [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
Name: genres, dtype: object

In [17]:
import pandas as pd
df_md = pd.read_csv('your_modified_file.csv')

In [18]:
df_md['genres'].head()

0    [28, 12, 14, 878]
1         [12, 14, 28]
2         [28, 12, 80]
3     [28, 80, 18, 53]
4        [28, 12, 878]
Name: genres, dtype: object

In [19]:
import pandas as pd
from ast import literal_eval


# It seems like the genres are already in a list format. If not, convert them:
df_md['genres'] = df_md['genres'].apply(lambda x: literal_eval(x) if isinstance(x, str) else x)

# Explode the genre_ids to have each genre ID in its own row
expanded_data = df_md.explode('genres')

# Rename 'genres' to 'genre_ids' if the names are causing confusion
expanded_data = expanded_data.rename(columns={'genres': 'genre_id'})

# Now, calculate the average rating for each genre
avg_ratings_per_genre = expanded_data.groupby('genre_id')['vote_average'].mean()

avg_ratings_per_genre


genre_id
12       6.156962
14       6.096698
16       6.341453
18       6.388594
27       5.626590
28       5.989515
35       5.945587
36       6.719797
37       6.178049
53       6.010989
80       6.274138
99       6.238182
878      6.005607
9648     6.183908
10402    6.355676
10749    6.207718
10751    6.029630
10752    6.713889
10769    6.352941
10770    5.662500
Name: vote_average, dtype: float64

In [20]:
genre_mapping = {
    28: "Action",
    12: "Adventure",
    16: "Animation",
    35: "Comedy",
    80: "Crime",
    99: "Documentary",
    18: "Drama",
    10751: "Family",
    14: "Fantasy",
    36: "History",
    27: "Horror",
    10402: "Music",
    9648: "Mystery",
    10749: "Romance",
    878: "Science Fiction",
    10770: "TV Movie",
    53: "Thriller",
    10752: "War",
    37: "Western",
    10769: "Foreign"
}


In [21]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog, Label
import requests
from PIL import Image, ImageTk

# API Key and Configuration
api_key = '28d241c9e0e99ba4cbb66b84c1f4f0d8'  # Replace with your actual TMDB API key
image_base_url = 'https://image.tmdb.org/t/p/w500'  # Base URL for fetching images

def fetch_movie_details(api_key, query):
    url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={query}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            return data['results'][0]  # Return the first result
        else:
            return None
    else:
        return None

def fetch_image(url, max_size=(300, 450)):
    """Fetch and resize an image to fit within the GUI."""
    try:
        response = requests.get(url)
        response.raise_for_status()  # Ensure the request was successful
        image = Image.open(requests.get(url, stream=True).raw)
        image.thumbnail(max_size, Image.ANTIALIAS)
        return ImageTk.PhotoImage(image)
    except Exception as e:
        print(f"Failed to load image: {e}")
        return None

def classify_movie(genre_ids, movie_rating):
    genre_ratings = [avg_ratings_per_genre.get(genre_id) for genre_id in genre_ids if genre_id in avg_ratings_per_genre]
    if not genre_ratings:
        return "No data to classify"

    average_genre_rating = sum(genre_ratings) / len(genre_ratings)
    threshold = 0.5
    if movie_rating > average_genre_rating + threshold:
        return "Good"
    elif movie_rating < average_genre_rating - threshold:
        return "Bad"
    else:
        return "Average"

def show_genre_average():
    selected_genre = genre_combobox.get()
    if not selected_genre:
        messagebox.showinfo("Error", "Please select a genre first.")
        return
    genre_id = int(selected_genre.split()[0])  # Extracting the genre ID
    avg_rating = avg_ratings_per_genre.get(genre_id, "No data")
    messagebox.showinfo("Average Rating", f"The average rating for {genre_mapping[genre_id]} is: {avg_rating:.2f}")

def search_movie():
    # Reset the GUI components
    genre_combobox.set('')
    genre_combobox['values'] = []
    label_details.config(text="No movie selected.")
    image_label.config(image='')  # Clear any previous image
    image_label.image = None  # Drop the reference to prevent memory leaks

    # Ask user for the movie name
    movie_name = simpledialog.askstring("Movie Search", "Enter the name of the movie:")
    if not movie_name:
        return  # If no movie name is given, just return

    # Fetch movie details from TMDB API
    movie_details = fetch_movie_details(api_key, movie_name)
    if movie_details:
        # Handle the genres for the fetched movie
        genres = movie_details['genre_ids']
        genre_combobox['values'] = [f"{genre_id} {genre_mapping.get(genre_id, 'Unknown')}" for genre_id in genres]
        if genres:  # Set the first genre as default selected in combobox
            genre_combobox.set(f"{genres[0]} {genre_mapping.get(genres[0], 'Unknown')}")

        # Update movie details
        movie_rating = movie_details['vote_average']
        classification = classify_movie(genres, movie_rating)
        details_text = f"{movie_details['title']} ({movie_details['release_date']})\n"
        details_text += f"Rating: {movie_rating}/10 from {movie_details['vote_count']} votes\n"
        details_text += f"Classification: {classification}"
        label_details.config(text=details_text)

        # Fetch and display the poster if available
        if movie_details.get('poster_path'):
            image_url = image_base_url + movie_details['poster_path']
            photo = fetch_image(image_url)
            if photo:
                image_label.config(image=photo)
                image_label.image = photo  # Keep a reference
    else:
        messagebox.showinfo("Search Result", "Movie not found or failed to fetch details.")

root = tk.Tk()
root.title("Movie Details and Genre Selection")

image_label = Label(root)  # Label for displaying the movie image
image_label.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

label_details = tk.Label(root, text="No movie selected.", justify=tk.LEFT)
label_details.pack(side=tk.TOP, fill=tk.X)

btn_search = tk.Button(root, text="Search Movie", command=search_movie)
btn_search.pack(side=tk.TOP, pady=10)

genre_label = tk.Label(root, text="Select which type of genre you are interested in:")
genre_label.pack(side=tk.TOP, fill=tk.X)

genre_combobox = ttk.Combobox(root, width = 50, state="readonly")
genre_combobox.pack(side=tk.TOP, fill=tk.X, padx=10, pady=10)

btn_show_avg = tk.Button(root, text="Show Genre Average", command=show_genre_average)
btn_show_avg.pack(side=tk.TOP, pady=10)

root.mainloop()


C:\Users\Srinivas\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:28: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
